# A note about opening notebooks in shared workspaces <a class="tocSkip">

Please do not run master copies of notebooks unless you intend to improve the code. As a general rule, it is good to be cautious when editing a notebook in a shared workspace, because you don't want to overwrite the work of your collaborators. Best practices is to test in a cloned workspace or make a duplicate notebook with an easily identifiable name.

# Notebook overview <a class="tocSkip">
    
Terra hosts several datasets stored in BigQuery, and this notebook gives an example of how to import BigQuery data into a notebook using the Data Explorer. We will be working with public-access, low coverage 1,000 Genomes Project data. 

This notebook is the last step in the Terra_Notebook_QuickStart workspace flow:   
![Notebooks_QuickStart_flow](https://storage.cloud.google.com/terra-featured-workspaces/QuickStart/Notebook_QuickStart_oval_flow.png)


This notebook assumes you have already done the following (using instructions in the [Terra_Notebooks_Quickstart](https://app.terra.bio/#workspaces/fc-product-demo/Terra_Notebooks_Quickstart/notebooks) workspace dashboard):   
- Create a data subset (cohort) in the Data Explorer
- Export the particpant IDs of the cohort to your cloned workspace data table

As you execute the code in the notebook, you'll go through the following steps: 

1. Set up the notebook virtual environment
2. Get the cohort query (participant IDs) into the notebook environment
3. Join data from BigQuery table to the participant IDs in the notebook
4. Look at a plot of the data
5. Gather provenance for reproducibility


> # What is BigQuery?<a class="tocSkip">

> BigQuery is the Google Cloud Storage solution for structured data (like a spreadsheet optimized for quick retieval of particular sections that you access with a "query". To learn more, see this five-minute video from Google [here](https://www.youtube.com/watch?v=aupC-Wj7XDY). Many datasets, including the public-access [1,000 Genomes Project](https://www.internationalgenome.org/about), are stored in BigQuery, for anyone to access. 

> Exporting the cohort in Terra adds a query to the workspace data table.


**Important note about cost**    
Although you are not paying for the data storage for this public-access dataset, querying the data carries a cost, paid to Google Cloud Storage. In the case of this notebook, it is only a few cents. 

# Environment setup

> Before moving on to the analysis, you will need to make sure your virtual notebook environment includes basic R packages and libraries. 

Be sure to run the notebook **`R environment setup`** in this workspace first.

## Install additional R packages

Many notebooks reference additional packages and libraries, which you will install in this section.  

In [ ]:
# The bigrquery package makes it easy to work with data stored in Google BigQuery by allowing you to query 
# BigQuery tables and retrieve metadata about your projects, datasets, tables, and jobs
library(bigrquery)

# ggplot2 is a library of integrated plotting functions
library(ggplot2)

Next we need to allow the notebook kernel, which is on a virtual machine, to speak to the workspace, which is mounted to a google bucket.     

> **(Optional) technical detail**: The program that does this, called FISS (Firecloud Service Selector), is a Python module that allows API (Application Programming Interface) calls from the notebook to the workspace. 

In [ ]:
# The reticulate package translates Python code into R code, so we can use some Python commands in this R notebook
library(reticulate)

# Update firecloud service selector module that provides API calls 
# (Application Programming Interface) from the notebook to the workspace
# API calls allow two applications to talk to each other
system("pip install --upgrade firecloud", intern = TRUE)

In [ ]:
# Import firecloud service selector modules using reticulate
# Reticulate is an an R library that allows python commands to be used in notebooks

os <- import("os")
firecloud <- import("firecloud")
fiss <- import("firecloud.fiss")

## Set project globals

> These next few cells set global variables, like the workspace name, automatically. This way, the notebook can be used in multiple workspaces with different project names and users without any manual adjustments.

In [ ]:
# Set the project variables

PROJECT <- Sys.getenv('GOOGLE_PROJECT')
WORKSPACE <- os$path$basename(os$path$dirname(os$getcwd()))

In [ ]:
# Set the project id of the clould project to bill for queries to BigQuery

BILLING_PROJECT_ID <- Sys.getenv('GOOGLE_PROJECT')

In [ ]:
# To use access BigQuery, you must first autheniticate, or verify your identity.
# This code cell does that. 

bigrquery::set_service_token(Ronaldo::getServiceAccountKey())

# Get the cohort query

> In this section, we are going to pull information from the "cohort" table that we created at the beginning of this exercisefrom into the notebook environment. We will use the workspace variables and a program called FISS (Firecloud Service Selector) to accomplish this. 

In [ ]:
# The FISS module that communicates between the notebook environment and Terra is in Python, so the first step is to 
# create Python versions of the workspace and project variables

workspace = r_to_py(WORKSPACE)
project = r_to_py(PROJECT)

In [ ]:
# Bring information from the "cohort" table into a notebook variable for the name of the table, "cohort_entity".
cohort_entity<- fiss$fapi$get_entities(project,workspace,"cohort")$json()

# Look at the first row in the table, finds the column with the header "query" and saves this information 
# as a variable "cohort_query"     
cohort_query <- cohort_entity[[1]][['attributes']][['query']]

In [ ]:
# Sanity check - This cell just prints out information, including the SQL query the notebook will use

cohort_entity

If you get an error when running the cell, try rerunning...

## A note about selecting from multiple workspace cohorts

> The command you just ran grabs the first cohort in your workspace data table. If there were multiple queries saved into your workspace table, the one you choose can be changed by updating the number in the double brackets in the code cell above.   

> For example, using [[2]] would point to the cohort in the second row of the table.    

# Call BigQuery

In [ ]:
# Execute the query and return all results into an in-memory table, "t", in R

t <- bigrquery::bq_project_query(
    BILLING_PROJECT_ID,
    cohort_query
)
tt <- bigrquery::bq_table_download(t)

In [ ]:
# Examine the output. This is a table of the participant IDs of your cohort. 

print(tt)

# Add data by "joining" with another table

> The query you just did returns **just the participant IDs** in a table. To do an analysis, you will want additional information from the BigQuery dataset. This section is an example of how to do this. 

In [ ]:
# Query the public-access data to grab pairs of participants IDs and Genders and put in a table (named "table") 
query <- '
SELECT
    DISTINCT participant_id,
    Gender
FROM
    `verily-public-data.human_genome_variants.1000_genomes_participant_info`
'
table_data <- bigrquery::bq_project_query(
    BILLING_PROJECT_ID,
    query
)
table <- bigrquery::bq_table_download(table_data)
dim(table)

In [ ]:
# Create a **new table**, `merged_table`, of the participant ID and Gender pairs for the participants in the cohort
merged_table <- merge(x = tt, y = table, by="participant_id", all.x = TRUE)

In [ ]:
# Sanity check - print out the dimensions of the merged table 
# It should have two columns and the number of rows = size of your cohort
dim(merged_table)

# Plot
> This last section is to confirm that you have the data you expect in your notebook environment. If you were doing an actual analysis, this is where you would include your R or Python analysis code. 

In [ ]:
grouped <- table(merged_table$Gender)
print(grouped)

g <- ggplot(merged_table, aes(Gender))
g + geom_bar()

# Provenance

In [ ]:
# Including this information allows you to easly go back and see the details of your notebook environment
# Provenance is also recommended as best practices for reproducible research

devtools::session_info()

Copyright 2019 The Broad Institute, Inc., Verily Life Sciences, LLC All rights reserved.

This software may be modified and distributed under the terms of the BSD license. See the LICENSE file for details.